# EIE

First, include some libraries

In [ ]:
# Run boilerplate code to set up environment

%run ../prelude.py

## Read matrices


In [ ]:
i = Tensor.fromUncompressed(["CHW"], 
                            [ 1, 2, 0, 3, 4, 0, 0, 7, 8])
f = Tensor.fromUncompressed(["M", "CHW"],
                            [[ 1, 2, 0, 3, 4, 0, 0, 7, 8],
                            [ 0, 1, 0, 0, 8, 0, 0, 7, 0],
                            [ 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [ 1, 0, 0, 3, 0, 0, 0, 2, 0],
                            [ 0, 0, 0, 2, 0, 0, 0, 4, 0],
                            [ 0, 0, 0, 0, 0, 0, 0, 0, 0],
                            [ 2, 4, 0, 6, 0, 0, 0, 14, 16],
                            [ 2, 0, 0, 2, 0, 0, 3, 0, 8]
                            ])

o_verify = Tensor.fromUncompressed(["M"],
                                   [143, 83, 0, 24, 34, 0, 254, 72])

displayTensor(i.setColor("blue"))
displayTensor(f.setColor("green"))
displayTensor(o_verify)
    

## Fully connected - Naive


In [ ]:
swapped_f = f.swapRanks()

o = Tensor(rank_ids=["M"])

canvas = createCanvas(i, swapped_f, o)

i_chw = i.getRoot()
f_chw = swapped_f.getRoot()
o_m = o.getRoot()

for chw, (i_val, f_m) in i_chw & f_chw:    
    for m, (o_ref, f_val) in  o_m << f_m:
        o_ref += i_val * f_val
        addFrame(canvas, (chw,), (chw, m), (m,))

displayTensor(o)
displayCanvas(canvas)


## Check result

In [ ]:
o_verify == o

## EIE Input

In [ ]:
PEs = 2
split_size = (len(f.getRoot())+1)//PEs

eie_f = f.splitEqual(split_size).swapRanks(depth=1)
displayTensor(eie_f)

eie_o_verify = o_verify.splitEqual(split_size)
eie_o_verify
displayTensor(eie_o_verify)

In [ ]:
o = Tensor(rank_ids=["M1", "M0"])

canvas = createCanvas(i, eie_f, o)

i_chw = i.getRoot()
f_m1 = eie_f.getRoot()
o_m1 = o.getRoot()

for chw, i_val in i_chw:
    for m1, (o_m0, f_chw) in o_m1 << f_m1:   # parallel
        f_m = f_chw.getPayload(chw)
        for m, (o_ref, f_val) in o_m0 << f_m:
            o_ref += i_val * f_val
            addFrame(canvas, (chw,), (m1, chw, m), (m1, m))

displayTensor(o)
displayCanvas(canvas)


## Check result

In [ ]:
displayTensor(eie_o_verify)
displayTensor(o)
eie_o_verify == o

## Testing area

For running alternative algorithms

In [ ]:
displayTensor(o)
o.getRoot().uncompress()